### Dataset
The dataset involves the composition data from materials project. Compositions of the existing, previously synthesized materials are described here. We will train a model that can generate a composition similar the previously synthesized materials.

In [ ]:
import pandas as pd
df = pd.read_csv('Example_data.csv')
df.head()

### Data Loader
Simple data loader that loads the composition. Note that we do not have any labels. Just the X. The goal of the GAN model is to create the a vector similar to real data.

In [ ]:
import pandas as pd
import torch
from time import time
from torch.utils.data import Dataset, DataLoader

class Data(Dataset):
  def __init__(self):
    df = pd.read_csv('Example_data.csv')
    self.Xs = torch.Tensor(df.iloc[:,1:].to_numpy())

  def __len__(self):
    return self.Xs.shape[0]

  def __getitem__(self, idx):
    x = self.Xs[idx,:]
    return idx, x

# Loading data
print('loading data...',end=''); t = time()
data = Data()
dataloader = DataLoader(data, batch_size=64, shuffle=True)
print('completed', time()-t,'sec')

torch.manual_seed(1234)

### Model
Here we have a simple model that calculates the composition using the softmax function. and also the disciminator which is used to create some abstract distribution of the data. We are trying to make the distribution from the fake and the real data similar.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Generator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main_module = nn.Sequential(
            # Z latent vector 128
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,118),
            nn.Softmax(dim=1)
        )
    def forward(self, x):
        return  self.main_module(x)

class Discriminator(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.main_module = nn.Sequential(
            nn.Linear(118,128),
            nn.SELU(),
            nn.Linear(128,1))
    def forward(self, x):
        return self.main_module(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#build model
D = Discriminator().to(device)
G = Generator().to(device)

### Using Model
This function contains the utilization of the model.
Note the training process. The training consists of two phases, where the Discriminator is trained first, followed by the generator. 

In [ ]:
from time import time
import random
import numpy as np
import torch.optim as optim
from torch import autograd

def calculate_gradient_penalty(D, xr, xf):
    eta = torch.rand(xr.shape[0],1).to(device=device)
    interpolated = eta * xr + (1 - eta) * xf
    interpolated.requires_grad_()
    prob_interpolated = D(interpolated)
    gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                            grad_outputs=torch.ones_like(prob_interpolated).to(device=device),
                            create_graph=True, retain_graph=True, only_inputs=True)[0]
    grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return grad_penalty

def use_model(data_loader, D,G, criterion, Doptimizer, Goptimizer, i_iter,  name = None):
  #switch to model mode
  G.train()
  D.train()
  D_losses = []
  G_losses = []
  WasDs = []
  for idxs, xr in data_loader: # loop for each batch
    ### 1. training discriminator
    z = torch.randn(xr.shape[0], 128)
    z = z.to(device=device)
    
    xr = xr.to(device=device)
    xf = G(z).detach()
      
    Dreal_loss = D(xr).mean()
    Dfake_loss = D(xf).mean()
    gp = calculate_gradient_penalty(D, xr, xf.detach())
      
    D_loss = - Dreal_loss + Dfake_loss + 10 * gp
      
    Doptimizer.zero_grad()
    D_loss.backward()
    Doptimizer.step()

    D_losses.append(float(D_loss.item()))
    WasDs.append(abs(float((Dreal_loss - Dfake_loss).item())))

    ### 2. training generator
    z = torch.randn(xr.shape[0], 128).to(device=device)
    xf = G(z)
    G_loss = - D(xf).mean()
      
    Goptimizer.zero_grad()
    G_loss.backward()
    Goptimizer.step()
    
    G_losses.append(float(G_loss.item()))

  return np.mean(D_losses),np.mean(G_losses),np.mean(WasDs)

### Training
Here is the code for training the model. You will have to train the model for 1000 epochs to get a reliable model.

In [ ]:
import torch.optim as optim

## Training
Doptimizer = optim.Adam(D.parameters(),lr=0.001)
Goptimizer = optim.Adam(G.parameters(),lr=0.001)

best_train_loss = float('inf')
for i_iter in range(1000): # epochs
  Dloss, Gloss, WD = use_model(dataloader,D,G,None,Doptimizer,Goptimizer,i_iter) # training model
  print('Train loss [%03d]: D_loss %10.2e G_loss %10.2e WasD %10.2e'%(i_iter, Dloss, Gloss, WD))

torch.save(D.state_dict(),'DW.pth.tar') # we save the data
torch.save(G.state_dict(),'GW.pth.tar') # we save the data

### Using the model
Here we generate a random composition that are similar to the realistic materials' composition. 

In [ ]:
import numpy as np
G.eval()
z = torch.randn(64, 128).to(device=device)
with torch.no_grad(): # it does not compute the gradient. so it's faster
  fake_xs = G(z)

chemical_symbols = [
    # 0
    'X',
    # 1
    'H', 'He',
    # 2
    'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
    # 3
    'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar',
    # 4
    'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr',
    # 5
    'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
    'In', 'Sn', 'Sb', 'Te', 'I', 'Xe',
    # 6
    'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy',
    'Ho', 'Er', 'Tm', 'Yb', 'Lu',
    'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Po', 'At', 'Rn',
    # 7
    'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk',
    'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr',
    'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc',
    'Lv', 'Ts', 'Og']
xs = np.array(fake_xs.tolist())
xs[xs <0.01] = 0
for j in range(xs.shape[0]):
  nonzero = np.where(xs[j,:])[0]
  s = ''
  for i in nonzero:
    s += chemical_symbols[i-1] + '%.2f'%xs[j,i] +' '
  print(s)

### Making Data
For your information, the code below was used to make the data

In [ ]:
from pymatgen.ext.matproj import MPRester
import numpy as np
from ase.data import chemical_symbols
from pymatgen.core import Composition
import csv

with MPRester("gv5swR0lIWsfLsGk") as m:
    docs = m.query({},{'pretty_formula':True,'icsd_ids':True})
    raw_data = [(doc['pretty_formula'],doc['icsd_ids']) for doc in docs]

new_data = {}
for f,v in raw_data:
    if len(v) != 0:
        c = Composition(f)
        new_data[str(c)] = dict(c.fractional_composition)

sym_map = {}
for i,s in enumerate(chemical_symbols):
    sym_map[s] = i -1

data = np.zeros((len(new_data),len(chemical_symbols)-1))
names = []
for i,(name,d) in enumerate(new_data.items()):
    names.append(name)
    for k,v in d.items():
        data[i,sym_map[str(k)]] = v

data_to_copy = [['formula']+chemical_symbols[1:]]
for n, r in zip(names,data):
    data_to_copy.append([n]+r.tolist())

with open('data.csv','w',newline='') as f:
    writer = csv.writer(f)
    for l in data_to_copy:
        writer.writerow(l)